In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.distributed.rpc as rpc
from torch import distributed as dist

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
epochs = 2000
eval_interval = 100
learning_rate = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

vocab_size = 65

torch.manual_seed(1337)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
# vocab_size = len(chars)

In [ ]:
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers

decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


# a, b = get_batch("train")
# print(a.shape, b.shape)
# print(a[0])
# print(b[0])

In [ ]:
class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class Embedding(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        return tok_emb + pos_emb  # (B,T,C)


class Output(nn.Module):
    def __init__(self):
        super().__init__()
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, x, targets=None):
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        return logits

In [ ]:
class FinalModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = Embedding()

        self.block1 = rpc.remote("worker1", Block, args=(n_embd, n_head))
        self.block2 = rpc.remote("worker2", Block, args=(n_embd, n_head))
        self.block3 = rpc.remote("worker1", Block, args=(n_embd, n_head))
        self.block4 = rpc.remote("worker2", Block, args=(n_embd, n_head))

        # self.blocks = [Block(n_embd, n_head=n_head) for _ in range(n_layer)]

        # self.blocks = nn.Sequential(
        #     *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        # )
        self.b = Output()

    def forward(self, x, targets=None):
        x = self.a(x)  # (B,T,C)

        # Send input to LanguageModel1 on worker1
        x = self.block4.rpc_sync().forward(
            self.block3.rpc_sync().forward(
                self.block2.rpc_sync().forward(self.block1.rpc_sync().forward(x))
            )
        )

        # for block in self.blocks:
        #     x = block(x)

        # x = self.blocks(x)

        logits = self.b(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [ ]:
import os


def run_master():
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"

    # Initialize the RPC framework on the master
    rpc.init_rpc("master", rank=0, world_size=3)

    # Instantiate the final model on the master
    model = FinalModel()

    m = model.to(device)
    print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

    for i, j in model.state_dict().items():
        print(i, j.shape)

    # generate from the model
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

    # --------------------------------------------------------------- #

    @torch.no_grad()
    def estimate_loss():
        out = {}
        model.eval()
        for split in ["train", "val"]:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        model.train()
        return out

    # create a PyTorch optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # --------------------------------------------------------------- #

    for iter in range(epochs):
        if iter % eval_interval == 0 or iter == epochs - 1:
            losses = estimate_loss()
            print(
                f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
            )

        xb, yb = get_batch("train")

        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # generate from the model
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

    # Shutdown the RPC framework after execution
    rpc.shutdown()


if __name__ == "__main__":
    run_master()

/opt/anaconda3/envs/pytorch/lib/python3.11/site-packages/torch/distributed/c10d_logger.py:79: FutureWarning: You are using a Backend <class 'torch.distributed.distributed_c10d.ProcessGroupGloo'> as a ProcessGroup. This usage is deprecated since PyTorch 2.0. Please use a public API of PyTorch Distributed instead.
  return func(*args, **kwargs)


0.010561 M parameters
a.token_embedding_table.weight torch.Size([65, 64])
a.position_embedding_table.weight torch.Size([32, 64])
b.ln_f.weight torch.Size([64])
b.ln_f.bias torch.Size([64])
b.lm_head.weight torch.Size([65, 64])
b.lm_head.bias torch.Size([65])

.r-&u'zcmpFUzjFjPo!X:mhMstBAP
;j;:;tuBA!SD;CvLm3Dj-hASCZcAPlJ,3lRpD!ULewPhfHwjSREI-YtMRpBz$xAO?zAr!Fx&E'3!qRvfsPoWwVRdg,eGCQAL.Pq?a,&MCT;V
VMjbvjK!E;Ct$vSwCqfB.'pPn,e w
,!H;'RdhSxLpD,Doq!!NhyIDjG,Beh:syC.X.;y-R:LCy:Gx;jOn&Xt&&Lpx;;US
RGzeqStjN'rA&qxNrkn MOLEIrXz'i,jvrpwy,&yV,gGDeei,RIIy;XMbU!ln'phJ&CYj
dDANPNG&UNIVhFRBvY3!C
&MRVxoCaJEsNya'x!gQkpH!H'NJCo
'dsg? rNBFnEAsM kiiODvvJSv!dkpwyyOFJk'-gI cvppcqaROX!tROpqMElaNiPWfRdO-OqKO'yq:hBCASJALx;hwix!IyrLF&KX.dKPKLFoK!NC.CeJOiGAJdGLHYiwFKxTha.M;Bo,3IZyynE&W
OA3!xFTgc;ki$Ne!::CSGLLk'GblPfe:3QhwLLGs$XxLbSAKBT,CDYHX&jyjnSFpz!C!XDFPrBIF?wITBYmSpA,ob
'WDP3b:y&c.jeFRfWo,f3HpmSrRASJkFMqmC
Zc:qR&:;RnMB' lB-ubnjdqX.PJqph
F'GiGRAfPn$MZsA:RO&dq-O.SrC'oX3XBAI3D3-L3YqGCqpAAIHvo!3
N,PLLJnhGNhbM&U?;p

In [ ]:
def dot_product(x, y):
    return sum(x_i * y_i for x_i, y_i in zip(x, y))